In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from copy import deepcopy

import gettsim
from gettsim import get_policies_for_date
from gettsim import compute_taxes_and_transfers
params_dict, policy_func_dict = get_policies_for_date("2020")


# Policy Functions Tutorial

If you are interested in modifying the German Tax and Transfer System, you can do so by changing the parameters of already existing fuctions (as we did in this (LINK) notebook), or you change the ```policy_func_dict``` itself by changing, adding or deleting functions to the dictionary.

### 1. Change an existing function

We illustrate how to change a function using an example. The steps will be the same for similar tasks. 

In the German System, there are some transfers for low-income families that can´t be received in combination. Per default, GETTSIM will always choose the most favorable transfers and set other transfers to zero. This means that the family/household might be entitled to receive these other transfers, but not in combination with the chosen, more benefitial transfers. This assumption could model the behavior of the households/families in a wrong way, if they do not always choose the optimal transfers (from a monetary perspective). E.g. there could be a social stigma connected to certain transfers or some people simply do not know about some of the available transfers. To account for these frictions, we can turn off this function of GETTSIM so that we see all the transfers a family/household is entitled to, even if the transfers can´t be received in combination. 

#### 1.1 Find the Function

Here we can look for the function that we want to change: https://gettsim.readthedocs.io/en/stable/functions.html 

#### 1.2 Define Changes to  the Function

After you found the function that you want to change, copy the source code from the website to your notebook and change it just as you like:

In [4]:
def arbeitsl_geld_2_m_hh(
    arbeitsl_geld_2_m_minus_eink_hh,
    wohngeld_vorrang_hh,
    kinderzuschlag_vorrang_hh,
    wohngeld_kinderzuschlag_vorrang_hh,
    rentner_in_hh,
):
    out = arbeitsl_geld_2_m_minus_eink_hh.clip(lower=0)
    cond = (
        #wohngeld_vorrang_hh
        #| kinderzuschlag_vorrang_hh
        #| wohngeld_kinderzuschlag_vorrang_hh
         rentner_in_hh
    )
    out.loc[cond] = 0
    return out

The lines of the cell above that start with "#" usually do the priority check as described above. With the hash, the lines become a comment and do not influence the code anymore.

#### 1.3 Add changed Function to Dictionary

Now we deepcopy the ```policy_func_dict``` and replace the function with the changed function which we defined before.

In [59]:
policy_func_dict_reformed = deepcopy(policy_func_dict)
policy_func_dict_reformed["arbeitsl_geld_2_m_hh"] = arbeitsl_geld_2_m_hh

Computations with the new ```policy_func_dict_reformed``` will now have the characteristic of showing the value of all available transfers without checking which ones can´t be received in combination and without choosing the most profitable combination. This is useful for further analysis. For example you could speculate which transfers Germans receive in reality and present this in GETTSIM. 

### 2. Add new function

#### 2.1 Define Function

One transfer that doesn´t exist, but is easy to implement in GETTSIM could be a unconditional basic income for every citizen. Every household receives the unconditional basic income multiplied by the size of the household. We set the unconditional basic income on 1000€: 

In [41]:
def basic_income_m_hh(hh_id):
     return hh_id.groupby(hh_id).size() * 1000
    

Maybe it can be helpful to use an existing function as a template and change the parameters and the body of the function accordingly. You can find suitable templates here: https://gettsim.readthedocs.io/en/stable/functions.html 

#### 2.2 Add Function to Dictionary

In [42]:
policy_func_dict_with_basic_income = deepcopy(policy_func_dict)
policy_func_dict_with_basic_income["basic_income_m_hh"] = basic_income_m_hh

#### 2.3 Check if it Works

We can define some exemplary data easily. All we need is a  ```p_id``` and a ```hh_id```. Everything else is irrelevant for this example, because the transfer does not depend on any conditions. As a result we should see 4000€, because the simulated household has four members.

In [49]:
data = pd.DataFrame(index=np.arange(0, 4), columns=('p_id', 'hh_id') )

for x in np.arange(0, 4):
    data.loc[x] = [x, "1"]
    
data

,p_id,hh_id
0,0,1
1,1,1
2,2,1
3,3,1


In [50]:
targets = [
    "basic_income_m_hh",
    ]

In [51]:
df = compute_taxes_and_transfers(
    data=data,
    user_functions=policy_func_dict_with_basic_income,
    params=params_dict,
    targets= targets,
)
df

0    4000
1    4000
2    4000
3    4000
Name: basic_income_m_hh, dtype: int64

Looks like we implemented our new transfer correctly!